In [1]:
import glob, os, re
import pandas as pd
from obspy import UTCDateTime
import tkinter as tk
from PIL import Image, ImageTk

In [2]:
class ImageDisplay:
    def __init__(self, root, image_paths):
        self.root = root
        self.image_paths = image_paths
        self.current_image_index = 0
        self.user_responses = []  # 用于存储用户的响应

        # 加载第一张图片
        self.load_image(self.current_image_index)

        # 绑定键盘事件
        self.root.bind('<Key>', self.on_key_press)

        # 创建按钮框架
        self.button_frame = tk.Frame(root)
        self.button_frame.pack()

        # 创建 "Yes" 按钮
        self.yes_button = tk.Button(self.button_frame, text="Yes (Left)", command=self.on_yes)
        self.yes_button.pack(side=tk.LEFT, padx=10)

        # 创建 "No" 按钮
        self.no_button = tk.Button(self.button_frame, text="No (Right)", command=self.on_no)
        self.no_button.pack(side=tk.RIGHT, padx=10)

    def load_image(self, index):
        image_path = self.image_paths[index]
        try:
            image = Image.open(image_path)
            self.photo = ImageTk.PhotoImage(image)
            if hasattr(self, 'label'):
                self.label.config(image=self.photo)
            else:
                self.label = tk.Label(self.root, image=self.photo)
                self.label.pack()
        except:
            print(image_path)
            pass



    def on_yes(self):
        # print("User likes the image!")
        self.user_responses.append(True)  # 将 True 写入列表
        self.current_image_index = (self.current_image_index + 1) % len(self.image_paths)  # 循环更新索引
        if self.current_image_index == 0:
            self.end_loop()  # 如果回到第一张图片，结束循环
        else:
            self.load_image(self.current_image_index)

    def on_no(self):
        # print("User does not like the image!")
        self.user_responses.append(False)  # 将 False 写入列表
        self.current_image_index = (self.current_image_index + 1) % len(self.image_paths)  # 循环更新索引
        if self.current_image_index == 0:
            self.end_loop()  # 如果回到第一张图片，结束循环
        else:
            self.load_image(self.current_image_index)
    def on_other(self):
        # print("User does not like the image!")
        self.user_responses.append('Other')  # 将 False 写入列表
        self.current_image_index = (self.current_image_index + 1) % len(self.image_paths)  # 循环更新索引
        if self.current_image_index == 0:
            self.end_loop()  # 如果回到第一张图片，结束循环
        else:
            self.load_image(self.current_image_index)
    def end_loop(self):
        # print("User responses:", self.user_responses)
        print('exit ~~~~')
        self.root.quit()  # 退出主循环

    def on_key_press(self, event):
        
        if event.keysym == 'Left':  # 按下 'y' 键表示 Yes
            self.on_yes()
        elif event.keysym == 'Right':  # 按下 'n' 键表示 No
            self.on_no()
        elif event.keysym == 'Up':  # 按下 'n' 键表示 No
            self.on_other()
        else:
            print(f"Pressed {event.keysym}")  # 打印按键信息


In [7]:
time = '230621'
path = f'../Results/Detection_fishcall_grunt/csv'
figpath = f'/Volumes/home/Research/Detect_fish/Taoyuan_fishcall_grunt/fig/{time}'

In [8]:

for csvpath in sorted(glob.glob(f'{path}/result_{time}.csv')):
    df = pd.read_csv(csvpath)
    
    imagelist = []
    for i in range(len(df)):
        begintmp = re.split('[(|)|,]',df['time_min_date'].values[i]); endtmp = re.split('[(|)|,]',df['time_max_date'].values[i])
        tr_begin = UTCDateTime(int(begintmp[1]),int(begintmp[2]),int(begintmp[3]),int(begintmp[4]),int(begintmp[5]),int(begintmp[6]),int(begintmp[7]))
        tr_end = UTCDateTime(int(endtmp[1]),int(endtmp[2]),int(endtmp[3]),int(endtmp[4]),int(endtmp[5]),int(endtmp[6]),int(endtmp[7]))
        
        yy = str(tr_begin.year).zfill(2)
        mm = str(tr_begin.month).zfill(2)
        dd = str(tr_begin.day).zfill(2)        
        hour = str(tr_begin.hour).zfill(2)
        minute = str(tr_begin.minute).zfill(2)
        second = str(tr_begin.second).zfill(2)
        date_time = f'{yy[2:]}{mm}{dd}_{hour}{minute}{second}'
        
        imagelist.append(f'{figpath}/{date_time}.png')
    print(len(imagelist))
    
root = tk.Tk()
root.title("Image Display")

# 图片路径列表
image_paths = imagelist 

# 创建 ImageDisplay 实例
image_display = ImageDisplay(root, image_paths)

# 运行主循环
root.mainloop()
response = image_display.user_responses
# 确保主循环结束后销毁窗口
root.destroy()

print('Saving file........... ')
tempdf = pd.DataFrame({'keep': response})
df__ = df.reset_index(drop=True)
df_combine = pd.concat([df__, tempdf], join='outer', axis=1) 
df_combine.to_csv(f"../Results/Detection_fishcall_grunt/manual_csv/{csvpath.rsplit('/')[-1]}",index=False)

1920
Pressed Shift_R
exit ~~~~
Saving file........... 
